# `Cubeviz`: weigh a black hole

In this demo we will (very roughly) measure the mass of a black hole with [JWST MIRI](https://jwst-docs.stsci.edu/jwst-mid-infrared-instrument) observations of [Stephan's Quintent](https://webbtelescope.org/news/first-images/gallery/zoomable-image-stephans-quintet) (PI: Klaus Pontoppidan, [JWST Early Release Observation Program 2732](https://www.stsci.edu/jwst/science-execution/program-information.html?id=2732)). We will load a spectral cube into [Cubeviz](https://jdaviz.readthedocs.io/en/latest/cubeviz/index.html) and measure physical quantities from the spectra.

We begin with the necessary imports:

In [ ]:
import os
import tempfile

import numpy as np

import astropy.units as u
from astropy.constants import G

from astroquery.mast import Observations
from regions import PixCoord, CirclePixelRegion

from jdaviz import Cubeviz

def set_line_style(
    helper, subset, line_width, line_opacity, viewer='spectrum-viewer'
):
    """
    This function tweaks a few line style settings to make the results
    more easily visible in this notebook.
    """
    plot_options = helper.plugins['Plot Options']
    plot_options.viewer = viewer
    plot_options.layer = subset
    plot_options.line_opacity = line_opacity
    plot_options.line_width = line_width

We could use `astroquery` to download the JWST observations from [MAST](https://mast.stsci.edu/), but here we load the file stored on the JWebbinar portal:

In [ ]:
# JWST/MIRI observations of NGC 7319 (also available on MAST):
data_path = "/home/shared/preloaded-fits/jdaviz_data/"
fn = "jw02732-c1001_t004_miri_ch3-shortmediumlong_s3d.fits"
miri_spectral_cube_path = os.path.join(data_path, fn)

Now we load the data into [Cubeviz](https://jdaviz.readthedocs.io/en/latest/cubeviz/index.html), and set the colormap scaling from the [Plot Options](https://jdaviz.readthedocs.io/en/latest/cubeviz/plugins.html#plot-options) plugin:

In [ ]:
# initialize Cubeviz:
cubeviz = Cubeviz()

# load the dataset:
cubeviz.load_data(miri_spectral_cube_path)

# use the Plot Options plugin to 
# set the colorscale in the "flux-viewer"
plot_options = cubeviz.plugins['Plot Options']
plot_options.stretch_function = 'arcsinh'
plot_options.stretch_vmin = 0
plot_options.stretch_vmax = 2500

# display interactive viewer in notebook:
cubeviz.show()

The following cells accomplish tasks can also be completed by interactively clicking around in the `Cubeviz` helper above. We include the code in this notebook so you can reproduce most steps exactly, and see examples for how to use the [plugin APIs](https://jdaviz.readthedocs.io/en/latest/cubeviz/plugins.html).


### Define spatial regions

Now let's define a spatial region from pixel coordinates in the "flux-viewer" in the upper left:

In [ ]:
# define a circular spatial region near the
# center of the galaxy:
central_region = CirclePixelRegion(
    center=PixCoord(x=26.5, y=26.5), 
    radius=2
)

# load the regions into Cubeviz:
cubeviz.load_regions([central_region])

# set collapsed spectrum line style:
set_line_style(
    cubeviz, 'Subset 1', line_width=3, line_opacity=0.9
)

`Cubeviz` will "collapse" (by summing) the spectral cube within the selected spatial region into another spectrum in the "spectrum-viewer", shown above in red.

### Spectral line analysis

We can measure the [velocity dispersion](https://en.wikipedia.org/wiki/Velocity_dispersion) in the central region by using the [Line Analysis](https://jdaviz.readthedocs.io/en/latest/specviz/plugins.html#line-analysis) plugin:

In [ ]:
line_analysis = cubeviz.plugins['Line Analysis']

available_spectral_subsets = len(line_analysis.spectral_subset.choices) - 1

if available_spectral_subsets:
    velocity_dispersions = []
    spatial_subset = 'Subset 1'
    line_analysis.spatial_subset = spatial_subset
    line_analysis.spectral_subset = 'Subset 2'

    line_analysis_results = line_analysis.get_results()

    # index 2 corresponds to the "Gaussian Sigma Width":
    sigma_width = line_analysis_results[2]
    sigma_width = float(sigma_width['result']) * u.Unit(sigma_width['unit'])

    # last index corresponds to the wavelength centroid:
    center = line_analysis_results[-1]
    centroid_wavelength = float(center['result']) * u.Unit(center['unit'])

    velocity_dispersion = (centroid_wavelength + sigma_width).to(
        u.km / u.s, u.doppler_relativistic(centroid_wavelength)
    )

    print(f"{spatial_subset} velocity dispersion = {velocity_dispersion:.1f}")

else: 
    raise Exception(
        'Please select a spectral region in Cubeviz '
        'centered on an emission line to '
        'continue with the tutorial.'
    )

We can compare this result to the velocity dispersion from [Nelson & Whittle (1995)](https://ui.adsabs.harvard.edu/abs/1995ApJS...99...67N/abstract), for example, who find $130 \pm 25 \text{ km s}^{-1}$ for this galaxy.

### Estimating mass with help from astropy units

We can estimate the mass of the central black hole with a so-called [M-$\sigma$ relation](https://en.wikipedia.org/wiki/M%E2%80%93sigma_relation), like this one used in [Woo & Urry (2002)](https://ui.adsabs.harvard.edu/abs/2002ApJ...579..530W/abstract):

$${\frac  {M}{10^{8}M_{\odot }}}\approx 1.349 \left({\frac  {\sigma }{200~{{\rm {km}}}~{{\rm {s}}}^{{-1}}}}\right)^{{4.02}}.$$

In [ ]:
bh_mass = 1.349e8 * u.M_sun * (velocity_dispersion / (200 * u.km / u.s)) ** 4.02
bh_mass

We can compare this mass with the one in the literature to find the relative error:

In [ ]:
# Woo & Urry (2002), Table 5: 
# https://ui.adsabs.harvard.edu/abs/2002ApJ...579..530W/abstract
log10_bh_mass_lit = 7.38

fractional_error = (bh_mass - u.M_sun * 10**log10_bh_mass_lit) / bh_mass
print(f'Relative error vs lit: {100 * fractional_error:.1f}%')

## How dense is the central region near the black hole?

We can estimate the mass in the central region from the relative shifts in the emission line centroids in narrow regions on either side of the central black hole. As the central region galaxy rotates, approaching areas have blue-shifted emission lines, while receding areas have red-shifted lines. Let's measure these centroid shifts in two areas on either side of the center, convert the centroid shifts to velocities, and solve for the mass enclosed between the two areas.

### Define two new spatial regions

In [ ]:
regions = [
    CirclePixelRegion(
        center=PixCoord(x=29.2, y=21.3), 
        radius=2
    ),
    CirclePixelRegion(
        center=PixCoord(x=25.2, y=30.3), 
        radius=2
    )
]

cubeviz.load_regions(regions)

### Measure line centroid shifts

The centroids of the emission lines Doppler shift in response to bulk rotation. We can see this directly in the spectrum viewer! Here we use the Line Analysis plugin to measure the blue/red shifts in one emission line:

In [ ]:
# use the Line Analysis plugin measure spectral lines:
line_analysis = cubeviz.plugins['Line Analysis']

# check that user has interactively selected a spectral subset:
available_spectral_subsets = len(line_analysis.spectral_subset.choices) - 1

if available_spectral_subsets:
    
    # measure the centroid of the spectral line 
    # in each spatial subset 
    wavelength_centroids = []
    line_analysis.spectral_subset = 'Subset 2'
    for subset in sorted(cubeviz.get_interactive_regions().keys())[-2:]:
        line_analysis.spatial_subset = subset

        center = line_analysis.get_results()[-1]
        centroid_wavelength = float(center['result']) * u.Unit(center['unit'])
        wavelength_centroids.append(centroid_wavelength)
        print(f"{subset} centroid = {centroid_wavelength:.5f}")

    wavelength_centroids = u.Quantity(wavelength_centroids)
else: 
    # raise an error if no spectral subset has been selected:
    raise Exception(
        'Please select a spectral region in Cubeviz '
        'centered on an emission line to '
        'continue with the tutorial.'
    )

### Find angular separation between two regions

The sky separation between the centers of the two spatial regions that we have selected can be extracted if we have both the pixel coordinates and the WCS for the image:

In [ ]:
def pixel_regions_to_sky_regions(regions, data):
    """
    Convert regions in pixel coordinates to 
    regions in sky coordinates from WCS.
    """
    return [
        region.to_sky(
            data.get_object().meta['_orig_wcs'].celestial
        )
        for region in regions
    ]

# get the Data object that represents these observations:
data_label = cubeviz.app.data_collection[0].label
data = cubeviz.app.get_data_from_viewer('flux-viewer', data_label)

# measure the sky separation between the centers of 
# the two spatial regions:
region_a, region_b = pixel_regions_to_sky_regions(regions, data)
angular_separation = region_a.center.separation(region_b.center).to(u.arcsec)
angular_separation

### Convert angular separation to distance

If we know the distance to the NGC 7319, we can convert the angular separation to a rough distance between spatial regions:

In [ ]:
# Bitsakis et al. (2011): 
# https://ui.adsabs.harvard.edu/abs/2011A%26A...533A.142B/abstract
ngc7319_distance = 98.1 * u.Mpc

# Reminder: s = r * theta
projected_distance_from_center = (
    ngc7319_distance * 
    angular_separation.to_value(u.rad)
).to(u.pc) / 2

projected_distance_from_center

### Estimate rotational velocity and enclosed mass

We can convert the wavelength shifts to corresponding velocities with [equivalencies](https://docs.astropy.org/en/stable/units/equivalencies.html) in `astropy.units`:

In [ ]:
rotation_velocity = wavelength_centroids[1].to(
    u.km / u.s, u.doppler_relativistic(wavelength_centroids.mean())
)

rotation_velocity

The minimum mass$^*$ within this central region of the galaxy can be found by assuming acceleration due to gravity provides the centripetal force

\begin{eqnarray}
\text{(gravitational accel.)} &=& \text{(centripetal accel.),}\\
g &=& a_c,\\
\frac{GM}{r^2} &=& \frac{v^2}{r}.
\end{eqnarray}

Rearranging, we find

$$M = \frac{rv^2}{G},$$

so let's combine the quantities we've derived in Cubeviz and from the literature to measure the enclosed mass:

In [ ]:
min_enclosed_mass = (projected_distance_from_center * rotation_velocity**2 / G).to(u.M_sun)
min_enclosed_mass

Wow, that's a lot of mass$^*$!

<p style='text-align: right; color: gray'>$^*$The enclosed mass from this oversimplified derivation is likely smaller than the true mass, because we haven't used the inclination of the rotating galaxy, which would affect the inferred mass by a factor like $\sin(i)$.</p>

Now let's compare: (a) the estimated enclosed mass in the center between the two spatial subsets, with (b) the black hole mass derived from the velocity dispersion:

In [ ]:
bh_mass_fraction = float(min_enclosed_mass / bh_mass)
print(
    f"There's enough mass in the inner {projected_distance_from_center:.0f} "
    f"to fill {bh_mass_fraction:.2f} black holes!"
)

What is the average density in this central region?

In [ ]:
volume = 4/3 * np.pi * projected_distance_from_center ** 3
density = min_enclosed_mass / volume
density

Despite being in the "dense" central region, there's not much more than a few stars per cubic-parsec.

***

### Bonus: Putting the spatial regions in context
  
Finally, let's contextualize the spatial regions that we selected above. Let's view the beautiful JWST/NIRCam image from this same Early Release Observation program in [Imviz](https://jdaviz.readthedocs.io/en/latest/imviz/index.html), and load the spatial regions onto it:

In [ ]:
from jdaviz import Imviz

# JWST/NIRCam observations of NGC 7319 (also available on MAST):
# Download the MIRI observations to a local temporary directory
fn_image = "jw02732-o001_t001_nircam_clear-f444w_i2d.fits"
nircam_image_path = os.path.join(data_path, fn)

imviz = Imviz()
imviz.load_data(nircam_image_path)
imviz.load_regions([region_a, region_b])

plot_options = imviz.plugins['Plot Options']

plot_options.image_colormap = 'viridis'
plot_options.stretch_function = 'log'
plot_options.stretch_vmin = 0.36
plot_options.stretch_vmax = 8
imviz.show()

viewer = imviz.default_viewer
viewer.center_on(region_a.center)

<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>